In [2]:
import tushare as ts

# 设置Tushare Token（需先注册获取）
ts.set_token('0bdd55bc3498291977bf4cb64a11848397643f62a1d917b7a149128d')
pro = ts.pro_api()

# 获取比亚迪股票日线数据
df = pro.daily(ts_code='002594.SZ', start_date='20220430', end_date='20250430')

# 保存为CSV文件
df.to_csv('BYD.csv', index=False)

In [39]:
import pandas as pd
import numpy as np

# 加载数据集
df = pd.read_csv(r'C:\Users\Ren\Desktop\MFE5210_assign\BYD.csv')

# 计算价格方向
df['Price_Direction'] = np.where(df['Clsprc'].pct_change() > 0, 1, -1)
print("Price_Direction 前几行：")
print(df['Price_Direction'].head())

# 分配上涨和下跌日的成交量
df['Up_Volume'] = df['Volume'].where(df['Price_Direction'] == 1, 0)
df['Down_Volume'] = df['Volume'].where(df['Price_Direction'] == -1, 0)
print("Up_Volume 和 Down_Volume 前几行：")
print(df[['Up_Volume', 'Down_Volume']].head())

# ---因子函数定义 ---

def calculate_momentum(df, window=20):
    """计算指定窗口期的动量，使用收盘价 Clsprc。"""
    df[f'Momentum_{window}D'] = df['Clsprc'].pct_change(window)
    return df

def calculate_volatility(df, window=20):
    """计算指定窗口期的波动率，使用收盘价 Clsprc。"""
    df[f'Volatility_{window}D'] = df['Clsprc'].pct_change().rolling(window).std()
    return df

def calculate_liquidity(df, window=20):
    """计算指定窗口期的流动性，使用成交额 Amount。"""
    df[f'Liquidity_{window}D'] = df['Amount'].rolling(window).mean() / 1e8  # 以亿为单位
    return df

def calculate_turnover_volatility(df, window=20):
    """计算指定窗口期的换手率波动率，使用 ToverOs。"""
    df[f'Turnover_Volatility_{window}D'] = df['ToverOs'].rolling(window).std()
    return df

def alpha6(df, window=10):
    """计算 Alpha6：开盘价与成交量的负相关性，指定窗口期。"""
    correlation = df['Opnprc'].rolling(window).corr(df['Volume'])
    df['Alpha6'] = -1 * correlation.fillna(0)
    return df

def calculate_vpr(df, window=5):
    """计算指定窗口期的成交量-价格共振 (VPR)。"""
    def calculate_slope(series):
        x = np.arange(len(series))
        slope = np.polyfit(x, series, 1)[0]
        return slope
    
    df[f'Volume_Slope_{window}D'] = df['Volume'].rolling(window).apply(calculate_slope, raw=True)
    df[f'Momentum_{window}D'] = df['Clsprc'].pct_change(window)
    df['VPR'] = np.where((df[f'Volume_Slope_{window}D'] > 0) & (df[f'Momentum_{window}D'] > 0), 1, 0)
    return df

def calculate_hlm(df, window=20):
    """计算指定窗口期的异质流动性动量 (HLM)。"""
    df[f'Up_Volume_Mean_{window}D'] = df['Up_Volume'].rolling(window).mean()
    df[f'Down_Volume_Mean_{window}D'] = df['Down_Volume'].rolling(window).mean()
    # 避免除零错误
    df[f'Volume_Ratio_{window}D'] = np.where(
        df[f'Down_Volume_Mean_{window}D'] != 0,
        df[f'Up_Volume_Mean_{window}D'] / df[f'Down_Volume_Mean_{window}D'],
        np.nan
    )
    df['HLM'] = np.sign(df[f'Momentum_{window}D']) * df[f'Volume_Ratio_{window}D']
    return df

def calculate_dvm(df, window=20):
    """计算指定窗口期的方向性成交量动量 (DVM)。"""
    df['Volume_Change_Dir'] = np.where(df['Volume'].pct_change() > 0, 1, -1)
    df['Direction_Match'] = (df['Price_Direction'] * df['Volume_Change_Dir'] > 0).astype(int)
    df['DVM'] = df['Direction_Match'].rolling(window).mean()
    return df

# --- 计算所有因子 ---

# 基础因子
df = calculate_momentum(df, window=20)
df = calculate_volatility(df, window=20)
df = calculate_liquidity(df, window=20)

# 换手率波动率
df = calculate_turnover_volatility(df, window=20)

# Alpha6
df = alpha6(df, window=10)

# 成交量-价格共振 (VPR)
df = calculate_vpr(df, window=5)

# 异质流动性动量 (HLM)
df = calculate_hlm(df, window=20)

# 方向性成交量动量 (DVM)
df = calculate_dvm(df, window=20)

print(df.head())
df.to_csv('BYD_factors.csv', index=False)

Price_Direction 前几行：
0   -1
1   -1
2   -1
3    1
4    1
Name: Price_Direction, dtype: int32
Up_Volume 和 Down_Volume 前几行：
   Up_Volume  Down_Volume
0          0     25325879
1          0     14548866
2          0     17511635
3   20969664            0
4   45952571            0
   Stock       Date  Opnprc   Hiprc   Loprc  Clsprc    Volume        Amount  \
0   2594   5/5/2022  246.20  252.39  241.77  248.80  25325879  6.282898e+09   
1   2594   5/6/2022  241.02  246.00  240.10  242.08  14548866  3.530097e+09   
2   2594   5/9/2022  231.00  236.89  231.00  232.73  17511635  4.086614e+09   
3   2594  5/10/2022  224.10  233.66  222.28  233.33  20969664  4.792959e+09   
4   2594  5/11/2022  232.00  256.66  231.50  252.70  45952571  1.148248e+10   

    ToverOs   OutShares  ...  Volume_Slope_5D  Momentum_5D  VPR  \
0  2.174335  1164764604  ...              NaN          NaN    0   
1  1.249082  1164764604  ...              NaN          NaN    0   
2  1.503448  1164764604  ...              NaN  

In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import os

# 设置 Matplotlib 后端为 'Agg'
plt.switch_backend('Agg')

# 加载数据集
try:
    df = pd.read_csv('BYD_factors.csv')
except FileNotFoundError:
    print("错误：未找到 'BYD_factors.csv' 文件，请确保文件在工作目录下。")
    exit(1)

# 选择因子列
factor_columns = ['Turnover_Volatility_20D', 'Alpha6', 'VPR', 'HLM', 'DVM']

# 检查因子列是否存在
missing_columns = [col for col in factor_columns if col not in df.columns]
if missing_columns:
    print(f"错误：以下因子列缺失：{missing_columns}")
    exit(1)

# 计算每日回报（基于 Clsprc）
df['Daily_Return'] = df['Clsprc'].pct_change()

# 检查数据缺失
print("\n因子列缺失值统计：")
print(df[factor_columns].isna().sum())
print("\nDaily_Return 缺失值：")
print(df['Daily_Return'].isna().sum())

# 删除缺失值（确保因子和回报数据完整）
df = df.dropna(subset=['Daily_Return'] + factor_columns, how='any')
print(f"\n清洗后数据行数：{len(df)}")

# --- 因子回测和指标计算函数 ---

def backtest_factor(df, factor):
    """
    对单一因子进行回测，计算 IC、夏普比率、收益风险比、最大回撤、年化收益率和年化波动率。
    投资信号：因子值 > 0 持有 BYD（Position=1），否则不持有（Position=0）。
    IC：因子值与下一交易日回报的 Spearman 相关系数。
    """
    # 计算 IC（Spearman 相关系数）
    forward_returns = df['Daily_Return'].shift(-1)  # 下一交易日回报
    ic, _ = spearmanr(df[factor], forward_returns, nan_policy='omit')
    
    # 基于因子值生成投资信号
    df['Position'] = np.where(df[factor] > 0, 1, 0)
    
    # 计算组合回报
    df['Portfolio_Return'] = df['Position'].shift(1) * df['Daily_Return']
    
    # 计算绩效指标
    portfolio_returns = df['Portfolio_Return'].dropna()
    mean_return = portfolio_returns.mean()
    std_return = portfolio_returns.std()
    
    # 夏普比率（假设无风险利率为 0，年度化）
    sharpe_ratio = mean_return / std_return * np.sqrt(252) if std_return != 0 else np.nan
    
    # 收益风险比
    profit_to_risk = mean_return / std_return if std_return != 0 else np.nan
    
    # 最大回撤
    cumulative_return = (1 + portfolio_returns).cumprod()
    rolling_max = cumulative_return.cummax()
    drawdowns = (rolling_max - cumulative_return) / rolling_max
    max_drawdown = drawdowns.max() if len(drawdowns) > 0 else np.nan
    
    # 年化收益率
    annualized_return = (1 + mean_return) ** 252 - 1 if mean_return is not np.nan else np.nan
    
    # 年化波动率
    annualized_volatility = std_return * np.sqrt(252) if std_return != 0 else np.nan
    
    return ic, sharpe_ratio, profit_to_risk, max_drawdown, annualized_return, annualized_volatility

# --- 计算因子绩效和指标 ---

performance_results = {}
for factor in factor_columns:
    ic, sharpe, profit_to_risk, max_drawdown, annualized_return, annualized_volatility = backtest_factor(df, factor)
    performance_results[factor] = {
        'IC': ic,
        'Sharpe_Ratio': sharpe,
        'Profit_to_Risk': profit_to_risk,
        'Max_Drawdown': max_drawdown,
        'Annualized_Return': annualized_return,
        'Annualized_Volatility': annualized_volatility
    }

# 转换为 DataFrame 显示
performance_df = pd.DataFrame(performance_results).T
print("\n因子绩效（包含 IC、最大回撤、年化收益率、年化波动率）：")
print(performance_df)

# --- 计算平均夏普比率 ---
average_sharpe = performance_df['Sharpe_Ratio'].mean()
print(f"\n平均夏普比率：{average_sharpe:.4f}")

# --- 计算相关系数矩阵 ---

correlation_matrix = df[factor_columns].corr()
print("\n相关系数矩阵：")
print(correlation_matrix)

# 检查相关系数矩阵是否有 NaN
print("\n相关系数矩阵 NaN 统计：")
print(correlation_matrix.isna().sum())

# 检查最大相关系数（忽略对角线）
max_corr = correlation_matrix.where(~np.eye(len(correlation_matrix), dtype=bool)).abs().max().max()
print(f"\n最大相关系数：{max_corr:.4f}")

# --- 绘制相关系数热力图 ---

try:
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title('Factor Correlation Matrix')
    output_path = 'factors_correlation_heatmap.png'
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.show() 
    plt.close()
    print(f"\n热力图已保存到：{os.path.abspath(output_path)}")
except Exception as e:
    print(f"\n错误：无法生成热力图，错误信息：{str(e)}")

# --- 检查因子数量和相关性 ---
print(f"\n因子数量：{len(factor_columns)}")
if max_corr <= 0.5:
    print("所有因子间的最大相关系数 ≤ 0.5，满足低相关性要求。")
else:
    print("存在因子间相关系数 > 0.5，建议调整因子定义或剔除高相关因子。")

# 保存绩效结果到 CSV
performance_df.to_csv('factor_performance.csv')
print("\n因子绩效已保存到：factor_performance.csv")


因子列缺失值统计：
Turnover_Volatility_20D    19
Alpha6                      0
VPR                         0
HLM                        20
DVM                        19
dtype: int64

Daily_Return 缺失值：
1

清洗后数据行数：707

因子绩效（包含 IC、最大回撤、年化收益率、年化波动率）：
                               IC  Sharpe_Ratio  Profit_to_Risk  Max_Drawdown  \
Turnover_Volatility_20D -0.017767      0.336500        0.021197      0.527581   
Alpha6                   0.029916      0.167668        0.010562      0.432310   
VPR                      0.029113      0.744909        0.046925      0.268274   
HLM                     -0.019992      0.265044        0.016696      0.352117   
DVM                     -0.015203      0.336500        0.021197      0.527581   

                         Annualized_Return  Annualized_Volatility  
Turnover_Volatility_20D           0.123265               0.345518  
Alpha6                            0.039089               0.228708  
VPR                               0.143744               0.180349  
HL